# 画像分類ハンズオン

## はじめに

本ノートブックでは，画像分類のビルトインアルゴリズムを使用して以下のことを行います。
- CIFAR10のデータセットを使ってモデルを転移学習する
- 転移学習したモデルでバッチ推論，リアルタイム推論する
- ハイパーパラメタをチューニングする
- チューニングしたモデルを使ってリアルタイム推論する

### 画像分類モデルの転移学習
本ノートブックでは，CIFAR10のデータセットを使って転移学習を行います。CIFAR10は，airplane（飛行機），automobile（自動車），bird（鳥），cat（猫），deer（鹿），dog（犬），frog（カエル），horse（馬），ship（船），truck（トラック）のいずれかが写っている 32x32画素の写真を集めてラベルをつけたデータセットです。 各クラスには6000 枚の画像が用意されており，学習用画像が全部で 50000 枚，テスト用画像が全部で 10000 枚あります。

## 下準備

### 権限や環境変数の設定

下準備として，以下の処理を行います。

* ロールの取得
* モデルなどのデータを格納するS3バケット名
* 画像分類のdockerコンテナを取得
* データセットのダウンロードと画像への変換
* lstファイルの作成
* 画像とlstファイルをS3にアップロード

**以下のセルの< BUCKET NAME >の部分を画像が格納されているS3バケット名に書き換えてください。**

**以下のセルの< USER NAME >の部分をご自分の名前など，同じアカウントを使用する他の方と区別しやすいものに書き換えてください。**

**以下のセルでは，学習と推論にml.p3.2xlargeを使用するよう設定していますが，ご自身のアカウントの状況によってはこのインスタンスを使用できないことがあります。その場合はご自身のアカウントの状況に合わせて最後の2行を書き換えてください。ただし，画像を使った学習を行うため，学習においてはp2やp3インスタンスを使用することが望ましいです。**

In [ ]:
%%time
import boto3
import time
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket = '< BUCKET NAME >'
username = '< USER NAME >'

training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

job_name_prefix = 'sagemaker-' + username
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())

train_instance_type = 'ml.p2.xlarge'
pred_instance_type = 'ml.m4.xlarge'

## データ取得
CIFAR10のデータセットを https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz からダウンロードします。

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar zvxf cifar-10-python.tar.gz

ダウンロードした tar.gz ファイルからデータセットの画像を取り出します。

CIFAR10 のデータセットには10クラス，60000枚の学習用画像がありますが，今回はデータセットから鳥，猫，犬の画像のみを使用します。学習用に各クラス500枚ずつ，検証用に各クラス50枚ずつの画像を使用します。

## データ前処理
取得したデータをモデルの入力形式に合わせるための前処理を行います。

画像分類のビルトインアルゴリズムは以下の入力形式に対応しています。今回はイメージ形式を使用するため、lstファイルを作成します。
* イメージ形式（lstファイルと画像）
* RecordIO形式
* 拡張マニフェスト形式（Amazon SageMaker Ground Truthの出力形式）

In [ ]:
!mkdir train
!mkdir validate
!mkdir train/bird
!mkdir train/cat
!mkdir train/dog
!mkdir validate/bird
!mkdir validate/cat
!mkdir validate/dog

import pandas as pd
import numpy as np
from PIL import Image

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

train_data_list = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
val_data_list = ['test_batch']

def save_as_jpg(mode, file_list, limit):
    bird_counter = 0
    cat_counter = 0
    dog_counter = 0
    limit_counter = 0
    
    for filepath in file_list:
        res = unpickle('cifar-10-batches-py/' + filepath)
        for idx, lb in enumerate(res[b'labels']):
            d = res[b'data'][idx].reshape([3, 32, 32])
            d = d.transpose(1, 2, 0)
            img = Image.fromarray(d)
            if lb == 2:
                if bird_counter > limit:
                    limit_counter += 1
                    continue
                img.save(mode + '/bird/bird_' + str(bird_counter).zfill(6) + '.jpg')
                bird_counter += 1
            elif lb == 3:
                if cat_counter > limit:
                    limit_counter += 1
                    continue
                img.save(mode + '/cat/cat_' + str(cat_counter).zfill(6) + '.jpg')
                cat_counter += 1
            elif lb == 5:
                if dog_counter > limit:
                    limit_counter += 1
                    continue
                img.save(mode + '/dog/dog_' + str(dog_counter).zfill(6) + '.jpg')
                dog_counter += 1
            if limit_counter == 3:
                break

save_as_jpg('train', train_data_list, 500)
save_as_jpg('validate', val_data_list, 50)

im2rec.pyをダウンロードし，これを使って学習用，検証用のlstファイルを作成します。

In [ ]:
%%bash

wget https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py -O im2rec.py
python im2rec.py --list --recursive cifar_train train/
python im2rec.py --list --recursive cifar_validate validate/

ローカルにある画像とlstファイルを SageMaker が使える状態にするために，S3 にアップロードします。<br>
まずはS3とローカルのパスを設定します。

In [ ]:
s3train = 's3://{}/{}/train/'.format(bucket, username)
s3validation = 's3://{}/{}/validation/'.format(bucket, username)
s3train_lst = 's3://{}/{}/train_lst/'.format(bucket, username)
s3validation_lst = 's3://{}/{}/val_lst/'.format(bucket, username)
val_lst_name = 'cifar_validate.lst'
train_lst_name = 'cifar_train.lst'
s3_train_lst_path = 's3://{}/{}/train_lst/{}'.format(bucket, username, train_lst_name)
s3_validation_lst_path = 's3://{}/{}/val_lst/{}'.format(bucket, username, val_lst_name)

local_validation_data_path = './validate/'
local_train_data_path = './train/'
object_categories = ['bird', 'cat', 'dog'] # 分類するクラスのラベル
batch_output_dir = 'cifar10'

awsコマンドで画像とlstファイルをS3にアップロードします。

In [ ]:
!aws s3 cp $local_train_data_path $s3train --recursive --quiet
!aws s3 cp $train_lst_name $s3_train_lst_path --quiet

!aws s3 cp $local_validation_data_path $s3validation --recursive --quiet
!aws s3 cp $val_lst_name $s3_validation_lst_path --quiet

のちほどモデルの精度を確認する際に使用するために，以下のセルを実行してvalidation用lstファイルを読み込みます。

In [ ]:
import pandas as pd

act_label = pd.read_csv(val_lst_name, header=None, sep='\t', names=['index', 'label', 'file'])
print(act_label[:5])

モデルを学習させる前に，学習パラメタを設定する必要があります。次のセクションではパラメタの詳細を説明します。

## 学習
データの準備ができたら、学習を行います。

### 学習パラメタの設定
以下に説明するようなアルゴリズム特有のハイパーパラメタを設定します。

* **num_layers**: ネットワークの層の数です。18, 34, 50, 101, 152,  200を使用できます。
* **image_shape**: 入力画像の次元を示し，'num_channels, height, width'の順で設定します。実際のサイズよりも大きい値を設定することはできません。チャネルの数は実際の画像と同じものにしてください。
* **num_training_samples**: 学習サンプルの総数です。caltechデータセットの場合は15240を指定します。今回は500枚＊3クラスで1500を設定します。
* **num_classes**: 新しいデータセットの分類クラス数です。Imagenetは1000のクラスに分類されますが，出力されるクラス数は転移学習の際に変えることができます。caltechデータセットの場合，クラス数はオブジェクトカテゴリ256種類＋カテゴリ外1で257を設定します。今回は鳥，猫，犬の3パターンを分類するので3を設定します。
* **mini_batch_size**: 各ミニバッチで使用する学習サンプルの数です。分散学習の場合，バッチごとに使用される学習サンプルの数はN＊mini_batchi_sizeとなります。Nは学習が実行されるホストの数です。
* **epochs**: 学習エポックの数です。
* **learning_rate**: 学習率です。
* **top_k**: 学習中のトップkの精度をリポートします。
* **resize**: 学習前に画像をリサイズします。画像は短辺がこの値になるようリサイズされます。値がセットされていない場合は，学習データはリサイズなしで使用されます。
* **checkpoint_frequency**: モデルパラメタを保存する頻度をエポック数で指定します。
* **use_pretrained_model**: 転移学習をする場合は1を設定します。

In [ ]:
# The algorithm supports multiple network depth (number of layers). They are 18, 34, 50, 101, 152 and 200
num_layers = 18
# we need to specify the input image shape for the training data
image_shape = "3,32,32"
# we also need to specify the number of training samples in the training set
num_training_samples = 1500
# specify the number of output classes
num_classes = 3
# batch size for training
mini_batch_size = 20
# number of epochs
epochs = 20
# learning rate
learning_rate = 0.01
# report top_k accuracy
top_k = 2
# period to store model parameters (in number of epochs), in this case, we will save parameters from epoch 2, 4, and 6
checkpoint_frequency = 2
# Since we are using transfer learning, we set use_pretrained_model to 1 so that weights can be 
# initialized with pre-trained weights
use_pretrained_model = 1

### SageMaker関連の学習パラメタの設定
それでは，学習を始めましょう。<br>
まずは学習インスタンスを立てるために必要なパラメタを設定します。

In [ ]:
import sagemaker

sess = sagemaker.Session()
s3_output_location = 's3://{}/{}/output'.format(bucket, username)
estimator = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                          train_instance_type=train_instance_type, 
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         base_job_name = job_name_prefix,
                                         sagemaker_session=sess)

### 機械学習アルゴリズム関連の学習パラメタの設定
次に，モデルのハイパーパラメタを設定します。

In [ ]:
estimator.set_hyperparameters(num_layers=num_layers, # レイヤー数
                             image_shape = image_shape,# 画像サイズ
                             num_classes=num_classes, # クラス数
                             num_training_samples=num_training_samples, # 学習データ数
                             mini_batch_size=mini_batch_size, # ミニバッチ(一度に学習させる単位)の画像枚数
                             epochs=epochs, # エポック数(同じデータを何回学習させるか)
                             learning_rate=learning_rate, # 学習率
                             top_k=top_k, # レポートされる精度の計算方法（上位何個のラベルのなかに答えのラベルがあれば正解とするか)
                             use_pretrained_model = use_pretrained_model,
                             precision_dtype='float32' # 計算時の精度。精度を落とすことで計算量が減る
                             )

### 入力データ形式の設定
次に，学習データと検証データを使用する準備をします。

In [ ]:
train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
train_lst = sagemaker.session.s3_input(s3train_lst, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')
validation_lst = sagemaker.session.s3_input(s3validation_lst, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')
 
data_channels = {'train': train_data, 'validation': validation_data, 'train_lst': train_lst, 'validation_lst': validation_lst}

### 学習ジョブ起動（学習開始）

fit関数で学習を実行します。<br><br>
ログに Completed - Training job completed と表示されたら学習の完了です。<br>
学習ジョブは以下のURLから参照可能です。<br>
注）以下のURLはバージニア北部リージョンのものです。他のリージョンをご利用の場合はリンクをクリック後にブラウザでリージョンを変更してください。<br>
https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs

In [ ]:
estimator.fit(inputs=data_channels, logs=True)

## 推論
モデルの学習が終わったら、学習済みモデルを使って推論を行います。<br>
SageMakerではバッチ推論とリアルタイム推論の２通りの推論方法の利用が可能です。
### バッチ推論
まずは推論を実行するたびに推論インスタンスを立てるバッチ推論を試してみましょう。

transform関数でバッチ推論を実行します。<br>
バッチ推論が完了するまで数分かかります。<br>
.が表示されたのち最後に！が出てきたらバッチ推論は完了です。

In [ ]:
batch_output_s3_dir = 's3://{}/{}/batch-inference/output/{}'
batch_output = batch_output_s3_dir.format(bucket, username, batch_output_dir)
transformer = estimator.transformer(instance_count=1, instance_type=pred_instance_type, output_path=batch_output)
transformer.transform(data=s3validation, data_type='S3Prefix', content_type='application/x-image')
transformer.wait()

バッチ推論結果は上のセルでtransformer関数を呼び出す際の `output_path` で指定された場所に保存されます。<br>
ここでは，S3に保存された推論結果ファイルを読み込み，推論結果を表示します。

In [ ]:
import json
import numpy as np
import os
import random

s3_client = boto3.client('s3')
def list_objects(s3_client, bucket, prefix):
    response = s3_client.list_objects(Bucket=bucket, Prefix=prefix)
    objects = [content['Key'] for content in response['Contents']]
    return objects

counter = 0
def get_label(s3_client, bucket, prefix):
    filename = prefix.split('/')[-1]
    s3_client.download_file(bucket, prefix, filename)
    with open(filename) as f:
        data = json.load(f)
        index = np.argmax(data['prediction'])
        probability = data['prediction'][index]
    
    global counter
    fname = os.path.basename(filename)[:-4]
    label = act_label[act_label['file'].str.contains(fname)]['label'].values[0]
    okng = 'NG'
    if label == index:
        okng = 'OK'
        counter += 1
    print("Result: " + okng + ", file - " + filename + ", label - " + object_categories[int(label)] + ", pred - " + object_categories[index] + ", probability - " + str(round(probability, 2)))
    return object_categories[index], probability

outputs = list_objects(s3_client, bucket, username + "/batch-inference/output/"+ batch_output_dir)


[get_label(s3_client, bucket, prefix) for prefix in random.sample(outputs, 20)]
print('accuracy - ' + str(round(counter/20, 2)))
!rm *.out

### リアルタイム推論
次に，リアルタイム推論を行います。<br>
まずはdeploy関数で推論エンドポイントを立てます。<br>
エンドポイントが立ち上がるまで数分かかります。

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=pred_instance_type)

predict関数で推論エンドポイントにひとつずつデータを送って推論を行います。

In [ ]:
import os
import glob
validation_image_list = sorted(glob.glob(os.path.join(local_validation_data_path, '**/*.jpg'), recursive=True))

In [ ]:
import random
def predict_images(predictor):
    counter = 0
    for file_name in random.sample(validation_image_list, 20):

        with open(file_name, 'rb') as f:
            payload = f.read()
            payload = bytearray(payload)
            
        response = predictor.predict(payload).decode()
        
        res_list = list(map(float, response[1:-1].split(',')))
        index = np.argmax(res_list)
        prob = max(res_list)

        fname = os.path.basename(file_name)
        label = act_label[act_label['file'].str.contains(fname)]['label'].values[0]
        okng = 'NG'
        if label == index:
            okng = 'OK'
            counter += 1
        print("Result: " + okng + ", image name - " + fname + ", label - " + object_categories[int(label)] + 
              ", pred - " + object_categories[index] + ", probability - " + str(round(prob, 2)) )
    print('accuracy - ' + str(round(counter/20, 2)))

In [ ]:
predict_images(predictor)

推論エンドポイントは起動している間課金対象となりますので，不要になったらすぐに削除することが推奨されます。

In [ ]:
estimator.delete_endpoint()

すでに学習済みのモデルがある場合，モデルが保存されているS3の場所を指定して以下のように推論エンドポイントを立てることが可能です。<br>
詳細は[自分で事前にトレーニングした MXNet または TensorFlow のモデルを Amazon SageMaker に導入する](https://aws.amazon.com/jp/blogs/news/bring-your-own-pre-trained-mxnet-or-tensorflow-models-into-amazon-sagemaker/)をご参照ください。

```python
sagemaker_model = MXNetModel(model_data = 's3://xxxx/model/model.tar.gz',
                             role = role,
                             entry_point = 'classifier.py')
predictor = sagemaker_model.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')
```

## ハイパーパラメタチューニング
ハイパーパラメータの値が適切でない場合，学習が収束しなかったり期待する精度にならなかったりすることがあります。<br>
ここからは，SageMakerによるハイパーパラメタチューニングの手順を学びます。

まず，ハイパーパラメタを探索する範囲を各ハイパーパラメタに対して設定します。

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'adam']),
                         'mini_batch_size': IntegerParameter(10, 64),
                         'learning_rate': ContinuousParameter(1e-4, 0.5),
                         'optimizer': CategoricalParameter(['sgd', 'adam', 'rmsprop', 'nag']),
                        'momentum': ContinuousParameter(0, 0.999),
                        'weight_decay': ContinuousParameter(0, 0.999),
                        'beta_1': ContinuousParameter(1e-4, 0.999),
                        'beta_2': ContinuousParameter(1e-4, 0.999),
                        'eps': ContinuousParameter(1e-4, 1.0),
                        'gamma': ContinuousParameter(1e-4, 0.999)}

次に，使用するメトリクスを設定します。ここでは，検証データの精度を使用します。

In [ ]:

objective_metric_name = 'validation:accuracy'
metric_definitions = [{'Name': 'Validation-accuracy',
                       'Regex': 'Validation-accuracy=([0-9\\.]+)'}]

次に，上で設定した値を使ってハイパーパラメタチューニングジョブの設定をします。

In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            base_tuning_job_name = job_name_prefix,
                            max_jobs=4,
                            max_parallel_jobs=2)

fit関数でハイパーパラメタチューニングジョブを実行します。

In [ ]:
tuner.fit({'train': train_data, 'validation': validation_data, 'train_lst': train_lst, 'validation_lst': validation_lst})

上のセルを実行すると，実行中であることを示すアスタリスクがすぐに数字になり実行が完了したかのように見えますが，実際はハイパーパラメタチューニングジョブが走っています。
以下のセルを実行して，ハイパーパラメタチューニングジョブのステータスを表示してみましょう。<br>
ステータスが InProgress から Completed になったらハイパーパラメタのチューニングが完了です。

ハイパーパラメタチューニングのステータスはこちらのURLから確認することも可能です。<br>
以下はバージニア北部のリージョンのURLです。他のリージョンを使用している場合は，以下のURLをクリックしたのちマネジメントコンソール右上のプルダウンメニューから適切なリージョンを選択してください。<br>
https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/hyper-tuning-jobs

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

チューニングしたモデルを使って推論用のエンドポイントを立てます。

In [ ]:
predictor = tuner.deploy(initial_instance_count=1, instance_type=pred_instance_type)

リアルタイム推論のときに作成した `predict_images` を使用して，validation用画像で推論して結果を表示します。

In [ ]:
predict_images(predictor)

## エンドポイントの削除
忘れずにエンドポイントを削除します。

In [ ]:
tuner.delete_endpoint()